OTU clustering 
====

* Picrust Requires a closed otu file generated by qiime with genegenes database ( either verison 13.5 or 13.8 )
* So below are the code which does the closed OTU picking 
* pick_closed_reference_otus.py makes a biom file
* filter the biom files if required to remove the potential eukaryotic stuff and singltons
* then add metadata to the files and upload it to phinch for graphical exploration

----

Tips
----

* run an open refernece otu picking also using pick_open_reference_otus.py
* The resulting biom file will have *de novo* OTUs alongside green genes OTUs 
* Filter out those OTUs which are unassigned and add meta data into using

----


Running Qiime for Closed OTU creation
----
```

echo "pick_otus:enable_rev_strand_match True"  >> $PWD/otu_picking_params_97.txt
echo "pick_otus:similarity 0.97" >> $PWD/otu_picking_params_97.txt

workon QiimeVenv

pick_closed_reference_otus.py -i $PWD/seqs.fna \
        -o $PWD/otu_clustering/ \
        -p $PWD/otu_picking_params_97.txt \
        -r $PWD/gg_13_5_otus/rep_set/97_otus.fasta \
        -a -O 20
    
deactivate


```

Add metadata to the biom file to port it to Phinch
----

```
biom add-metadata -i otu_table_mc2_w_tax.biom \
        -o otu_table_mc2_w_tax_and_metadata.json.biom \
        -m mapping_file.txt
```

Tips:
-----
* Make duplicate copy of the #SampleID column in mapping_file.txt and rename the column as PhichID
* Use this file biom add-metadata
* This will help in labels in Phinch

## Filtering out the otu biom file

#### filter choloplast , mt dna
```
filter_taxa_from_otu_table.py -i otu_table.biom \
        -o closed_ref_97_otu_table_no_euks.biom \
        -n cChloroplast,fmitochondria
        
        
filter_taxa_from_otu_table.py -i closed_ref_97_otu_table_no_euks.biom \
        -o closed_ref_97_otu_table_no_euks_no_unassigned.biom \
        -n Unassigned
```
#### singlton removal
```
filter_otus_from_otu_table.py -i closed_ref_97_otu_table_no_euks_no_unassigned.biom \
        -o closed_ref_97_otu_table_filtered.biom  \
        -n 2
```
#### biom file conversion and change taxonomy to Consensus Lineage

```
biom convert -i closed_ref_97_otu_table_filtered.biom \
        -o closed_ref_97_otu_table_filtered.json --to-json 
        
biom convert -i closed_ref_97_otu_table_filtered.biom \
        -o closed_ref_97_otu_table_filtered.tsv \
        --header-key taxonomy --to-tsv 
        
        
biom summarize-table -i closed_ref_97_otu_table_filtered.biom \
        -o closed_ref_97_otu_table_filtered.summary 
        
sed s'/taxonomy/Consensus Lineage/' -i closed_ref_97_otu_table_filtered.tsv

```